In [70]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from numpy import average
from matplotlib.pyplot import figure
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [71]:
#import CSVs, assign headers and remove quotations from column headers
red_wine = pd.read_csv(r'C:\Users\Killian\Projects\Wine analysis\Wine data\winequality-red.csv', sep=';',header=0, engine='python')
white_wine = pd.read_csv(r'C:\Users\Killian\Projects\Wine analysis\Wine data\winequality-white.csv', sep=';',header=0, engine='python')


In [72]:
red_wine['quality_label'] = red_wine['quality'].apply(lambda value: 'low'
if value <= 4 else 'medium-low'
if value == 5 else 'medium' if value == 6 else 'medium-high' if value == 7 else 'high')

white_wine['quality_label'] = white_wine['quality'].apply(lambda value: 'low'
if value <= 4 else 'medium-low'
if value == 5 else 'medium' if value == 6 else 'medium-high' if value == 7 else 'high')

red_wine['quality_label'] = pd.Categorical(red_wine['quality_label'],
categories=['low', 'medium-low', 'medium', 'medium-high', 'high'])

white_wine['quality_label'] = pd.Categorical(white_wine['quality_label'],
categories=['low', 'medium-low', 'medium', 'medium-high', 'high'])

In [84]:
#Create a new dataset combining red and white wines
red_wine['wine_type'] = 'red'
white_wine['wine_type'] = 'white'
wines = pd.concat([red_wine, white_wine])
wines['ql_code'] = wines['quality_label']

# re-shuffle records just to randomize data points as you may have a problem when fitting machine learning model. Dataset may not be split in a fair way.
wines = wines.sample(frac=1, random_state=42).reset_index(drop=True)

In [82]:
# Skewness is the degree of distortion from the symmetrical bell curve or the normal distribution. It measures the lack of symmetry in data distribution.
# It differentiates extreme values in one versus the other tail. A symmetrical distribution will have a skewness of 0.
# If the skewness is between -1 and -0.5(negatively skewed) or between 0.5 and 1(positively skewed), the data are moderately skewed. 
# If the skewness is less than -1(negatively skewed) or greater than 1(positively skewed), the data are highly skewed.

# Kurtosis is all about the tails of the distribution — not the peakedness or flatness. It is used to describe the extreme values
# in one versus the other tail. It is actually the measure of outliers present in the distribution.

# Mesokurtic: This distribution has kurtosis statistic similar to that of the normal distribution. It means that the extreme values of the 
# distribution are similar to that of a normal distribution characteristic. This definition is used so that the standard normal distribution has a kurtosis of three.

# Leptokurtic (Kurtosis > 3): Distribution is longer, tails are fatter. Peak is higher and sharper than Mesokurtic, which means that data are heavy-tailed or profusion 
# of outliers. Outliers stretch the horizontal axis of the histogram graph, which makes the bulk of the data appear in a narrow (“skinny”) vertical range, thereby giving 
# the “skinniness” of a leptokurtic distribution.

# Platykurtic: (Kurtosis < 3): Distribution is shorter, tails are thinner than the normal distribution. The peak is lower and broader than Mesokurtic, which means that 
# data are light-tailed or lack of outliers. The reason for this is because the extreme values are less than that of the normal distribution.


wine = []
for column in wines.columns[0:-4]:
        wine.append([wines[column].astype(float).skew().round(2), wines[column].astype(float).kurt().round(2)])

wine_stats = pd.DataFrame(wine, index=[wines.columns[0:-4]], columns=['Skewness', 'Kurtosis'])
display(wine_stats)

,Skewness,Kurtosis
fixed acidity,1.72,5.06
volatile acidity,1.50,2.83
citric acid,0.47,2.40
residual sugar,1.44,4.36
chlorides,5.40,50.90
free sulfur dioxide,1.22,7.91
total sulfur dioxide,-0.00,-0.37
density,0.50,6.61
pH,0.39,0.37
sulphates,1.80,8.65


In [85]:
enc = OrdinalEncoder(categories=[['low', 'medium-low', 'medium', 'medium-high', 'high']])

train_wine=wines

train_wine[['ql_code']] = enc.fit_transform(train_wine[['ql_code']])

train_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_label,wine_type,ql_code
0,7.0,0.17,0.74,12.80,0.045,24.0,126.0,0.99420,3.26,0.38,12.2,8,high,white,4.0
1,7.7,0.64,0.21,2.20,0.077,32.0,133.0,0.99560,3.27,0.45,9.9,5,medium-low,red,1.0
2,6.8,0.39,0.34,7.40,0.020,38.0,133.0,0.99212,3.18,0.44,12.0,7,medium-high,white,3.0
3,6.3,0.28,0.47,11.20,0.040,61.0,183.0,0.99592,3.12,0.51,9.5,6,medium,white,2.0
4,7.4,0.35,0.20,13.90,0.054,63.0,229.0,0.99888,3.11,0.50,8.9,6,medium,white,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,7.6,0.32,0.58,16.75,0.050,43.0,163.0,0.99990,3.15,0.54,9.2,5,medium-low,white,1.0
6493,5.6,0.28,0.27,3.90,0.043,52.0,158.0,0.99202,3.35,0.44,10.7,7,medium-high,white,3.0
6494,6.4,0.37,0.20,5.60,0.117,61.0,183.0,0.99459,3.24,0.43,9.5,5,medium-low,white,1.0
6495,6.5,0.26,0.50,8.00,0.051,46.0,197.0,0.99536,3.18,0.47,9.5,5,medium-low,white,1.0


In [86]:
# I will keep all features for now. Will remove low correlation features later to see if there is an impact

wine_predict=train_wine.quality_label
train_wine.drop(['quality_label'],axis=1,inplace=True)
print(wine_predict)

train_wine_train, train_wine_test, wine_predict_train, wine_predict_test = train_test_split(train_wine, wine_predict, test_size=0.2, random_state=42)

0              high
1        medium-low
2       medium-high
3            medium
4            medium
           ...     
6492     medium-low
6493    medium-high
6494     medium-low
6495     medium-low
6496     medium-low
Name: quality_label, Length: 6497, dtype: category
Categories (5, object): ['low', 'medium-low', 'medium', 'medium-high', 'high']
